In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import selenium
from selenium import webdriver
from urllib.request import urlopen
from selenium.webdriver.common.by import By
import re
import time
from tqdm import tqdm_notebook

In [2]:
def printRanking(lst1, lst2, lst3, lst4):
    idx = 1
    for item in zip(lst1, lst2, lst3, lst4):
        print(f"{idx}. {item[0]}, 평점: {item[1]}, 예매율: {item[2]}, 개봉일: {item[3]}")
        idx += 1

In [3]:
def daumRanking():
    url = 'https://movie.daum.net/ranking/reservation'
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'lxml')

    ol = soup.find('ol')

    tit_items = ol.find_all('div', class_='thumb_cont')

    titlelst = []
    ratinglst = []
    bookinglst = []
    opening_datelst = []
    for item in tit_items:
        title = item.find('a', attrs = {'class':'link_txt'}).text
        rating, booking = item.find('span', attrs = {'class':'txt_append'}).text.strip().split('\n')
        date = item.find('span', attrs = {'class':'txt_info'}).text.strip()

        titlelst.append(title)
        ratinglst.append(rating[2:])
        bookinglst.append(booking[3:])
        opening_datelst.append(date[2:])

    printRanking(titlelst, ratinglst, bookinglst, opening_datelst)

In [4]:
daumRanking()

1. 블랙 아담, 평점: 7.1, 예매율: 23.6%, 개봉일: 22.10.19
2. 리멤버, 평점: 8.6, 예매율: 16.5%, 개봉일: 22.10.26
3. 자백, 평점: 6.7, 예매율: 14.8%, 개봉일: 22.10.26
4. 에브리씽 에브리웨어 올 앳 원스, 평점: 6.9, 예매율: 6.5%, 개봉일: 22.10.12
5. 인생은 아름다워, 평점: 9.0, 예매율: 5.6%, 개봉일: 22.09.28
6. 공조2: 인터내셔날, 평점: 7.9, 예매율: 5.0%, 개봉일: 22.09.07
7. 극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교, 평점: 8.6, 예매율: 3.6%, 개봉일: 22.09.28
8. 귀못, 평점: 8.4, 예매율: 2.7%, 개봉일: 22.10.19
9. 오펀: 천사의 탄생, 평점: 6.6, 예매율: 2.3%, 개봉일: 22.10.12
10. 정직한 후보 2, 평점: 3.4, 예매율: 1.3%, 개봉일: 22.09.28
11. 스타게이저: 아스트로스코프, 평점: 0.0, 예매율: 1.2%, 개봉일: 22.10.27
12. 스마일, 평점: 7.2, 예매율: 1.1%, 개봉일: 22.10.06
13. 극장판 헬로카봇: 수상한 마술단의 비밀, 평점: 9.1, 예매율: 0.8%, 개봉일: 22.09.28
14. 미혹, 평점: 7.4, 예매율: 0.6%, 개봉일: 22.10.19
15. 티켓 투 파라다이스, 평점: 7.6, 예매율: 0.6%, 개봉일: 22.10.12
16. 낮과 달, 평점: 8.6, 예매율: 0.6%, 개봉일: 22.10.20
17. 고속도로 가족, 평점: 10.0, 예매율: 0.5%, 개봉일: 22.11.02
18. 대무가, 평점: 7.3, 예매율: 0.4%, 개봉일: 22.10.12
19. 더 요트, 평점: 5.7, 예매율: 0.4%, 개봉일: 22.10.20
20. 옆집사람, 평점: 10.0, 예매율: 0.4%, 개봉일: 22.11.03


In [5]:
import pandas as pd

In [55]:
date = pd.date_range('2021-5-1', periods=20, freq='W')
print(len(date))
date


20


DatetimeIndex(['2021-05-02', '2021-05-09', '2021-05-16', '2021-05-23',
               '2021-05-30', '2021-06-06', '2021-06-13', '2021-06-20',
               '2021-06-27', '2021-07-04', '2021-07-11', '2021-07-18',
               '2021-07-25', '2021-08-01', '2021-08-08', '2021-08-15',
               '2021-08-22', '2021-08-29', '2021-09-05', '2021-09-12'],
              dtype='datetime64[ns]', freq='W-SUN')

In [63]:
movie_date = []
movie_name = []
movie_opening = []
movie_attendance = []

In [64]:
reg = re.compile(r'([0-9]+.[0-9]+.[0-9]+)')

for today in tqdm_notebook(date):
    html = "https://movie.daum.net/ranking/boxoffice/weekly?date={date}"
    response = urlopen(html.format(date=urllib.parse.quote(today.strftime('%Y%m%d'))))
    soup = BeautifulSoup(response, "lxml")
    ol = soup.find('ol')
    tit_items = ol.find_all('div', class_='thumb_cont')
    end = len(tit_items)
    for item in tit_items:
        title = item.find('a', attrs = {'class':'link_txt'}).text
        txtinfo = item.find('span', attrs = {'class':'txt_info'})
        movie_name.append(title)
        
    regfinds = re.findall(reg, str(soup))
    for i in regfinds:
        movie_opening.append(i[0])
        movie_attendance.append(i[1])
            
    movie_date.extend([today for n in range(0, end)])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11940\7374863.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for today in tqdm_notebook(date):


  0%|          | 0/20 [00:00<?, ?it/s]

In [65]:
len(movie_date), len(movie_name), len(movie_opening), len(movie_attendance)

(300, 300, 2383, 2383)